In [17]:
import yfinance as yf
import numpy as np
import pandas as pd
from stocktrends import Renko



In [18]:
tickers = ["AAPL", "GOOG", "META"]
ohlcv_data = {}
hour_data ={}
renko_data ={}

In [19]:
for ticker in tickers:
    temp = yf.download(ticker, period = '1mo', interval = '15m')
    temp.dropna(how = 'any', inplace = True)
    ohlcv_data[ticker] = temp
    temp = yf.download(ticker, period = '1y', interval = '1h')
    temp.dropna(how = 'any', inplace = True)
    hour_data[ticker] = temp

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

c:\Users\IVAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\yfinance\utils.py:633: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if not _np.isnan(quotes["High"][n - 1]):
c:\Users\IVAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\yfinance\utils.py:634: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  quotes.loc[idx2, "High"] = _np.nanmax([quotes["High"][n - 1], quotes["High"][n - 2]])
c:\Users\IVAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\yfinance\utils.py:638: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will alway


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

c:\Users\IVAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\yfinance\utils.py:633: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if not _np.isnan(quotes["High"][n - 1]):
c:\Users\IVAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\yfinance\utils.py:634: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  quotes.loc[idx2, "High"] = _np.nanmax([quotes["High"][n - 1], quotes["High"][n - 2]])
c:\Users\IVAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\yfinance\utils.py:638: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will alway


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


c:\Users\IVAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\yfinance\utils.py:633: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if not _np.isnan(quotes["High"][n - 1]):
c:\Users\IVAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\yfinance\utils.py:634: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  quotes.loc[idx2, "High"] = _np.nanmax([quotes["High"][n - 1], quotes["High"][n - 2]])
c:\Users\IVAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\yfinance\utils.py:638: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will alway

In [20]:
def ATR(DF, n=14):
    "function to calculate True Range and Average True Range"
    df = DF.copy()
    df["H-L"] = df["High"] - df["Low"]
    df["H-PC"] = abs(df["High"] - df["Adj Close"].shift(1))
    df["L-PC"] = abs(df["Low"] - df["Adj Close"].shift(1))
    df["TR"] = df[["H-L","H-PC","L-PC"]].max(axis=1, skipna=False)
    df["ATR"] = df["TR"].ewm(com=n, min_periods=n).mean()
    return df["ATR"]

In [21]:
def renko_DF(DF, hourly_df):
    df = DF.copy()
    df.drop("Close", axis = 1, inplace = True)
    df.reset_index(inplace = True)
    df.columns = ["date", "open", "high","low","close","volume"]
    df2 = Renko(df)
    df2.brick_size = 3*round(ATR(hourly_df, 120).iloc[-1],0)
    renko_df = df2.get_ohlc_data()
    return renko_df

for ticker in ohlcv_data:
    renko_data[ticker] = renko_DF(ohlcv_data[ticker], hour_data[ticker])
    